In [ ]:
import sys
print(sys.path)

In [ ]:
import argparse
import base64
import json
import cv2

import numpy as np
import socketio
import eventlet
import eventlet.wsgi
import time
import math
from PIL import Image
from PIL import ImageOps
from flask import Flask, render_template
from io import BytesIO

from keras.models import model_from_json
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array
import tensorflow as tf
#tf.python.control_flow_ops = tf

new_size_col = 200
new_size_row = 66

def cropImage(image):
    # Preprocessing image files
    shape = image.shape
    # note: numpy arrays are (row, col)!
    image = image[60:shape[0]-20, 10:shape[1]-10]
#     image = cv2.resize(image,(new_size_col,new_size_row), interpolation=cv2.INTER_AREA)    
    return image

def resizeImage(image):
    # Preprocessing image files
    shape = image.shape
    image = cv2.resize(image,(new_size_col,new_size_row), interpolation=cv2.INTER_AREA)   
    return image

def preprocess(image):
    
    
    # get shape and chop off 1/3 from the top
    image = cropImage(image)
    image = resizeImage(image)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)
    # note: numpy arrays are (row, col)!
    #image = image[shape[0]-70:shape[0]-25, 0:shape[1]]
    #image = cv2.resize(image, (200,66), interpolation=cv2.INTER_AREA)
    #image = cv2.resize(image, (64,64), interpolation=cv2.INTER_AREA)
    return image



sio = socketio.Server()
app = Flask(__name__)
model = None
prev_image_array = None





class SimplePIController:
    def __init__(self, Kp, Ki):
        self.Kp = Kp
        self.Ki = Ki
        self.set_point = 0.
        self.error = 0.
        self.integral = 0.

    def set_desired(self, desired):
        self.set_point = desired

    def update(self, measurement):
        # proportional error
        self.error = self.set_point - measurement

        # integral error
        self.integral += self.error
        
        cal_throttle = self.Kp * self.error + self.Ki * self.integral
        throttle = min(cal_throttle, 0.5)
        
        return throttle


controller = SimplePIController(0.1, 0.002)
set_speed = 20
controller.set_desired(set_speed)



@sio.on('telemetry')
def telemetry(sid, data):
    # The current steering angle of the car
    steering_angle = data["steering_angle"]
    # The current throttle of the car
    throttle = data["throttle"]
    # The current speed of the car
    speed = data["speed"]
    # The current image from the center camera of the car
    imgString = data["image"]
    image = Image.open(BytesIO(base64.b64decode(imgString)))
    image_pre = np.asarray(image)
    image_array = preprocess(image_pre)
    #image_array = np.asarray(image)
    transformed_image_array = image_array[None, :, :, :]
    # This model currently assumes that the features of the model are just the images. Feel free to change this.
    steering_angle = 1.0*float(model.predict(transformed_image_array, batch_size=1))
    # The driving model currently just outputs a constant throttle. Feel free to edit this.
#     throttle = controller.update(float(speed))
    if float(speed) > 20:
        throttle = 0.1
    else:
        throttle = 0.3
    print(steering_angle, throttle)
    send_control(steering_angle, throttle)
    
    # save frame
#     if args.image_folder != '':
#         timestamp = datetime.utcnow().strftime('%Y_%m_%d_%H_%M_%S_%f')[:-3]
#         image_filename = os.path.join(args.image_folder, timestamp)
#         image.save('{}.jpg'.format(image_filename))
#     else:
#         # NOTE: DON'T EDIT THIS.
#         sio.emit('manual', data={}, skip_sid=True)


@sio.on('connect')
def connect(sid, environ):
    print("connect ", sid)
    send_control(0, 0)


def send_control(steering_angle, throttle):
    sio.emit("steer", data={
    'steering_angle': steering_angle.__str__(),
    'throttle': throttle.__str__()
    }, skip_sid=True)


if __name__ == '__main__':
#     parser = argparse.ArgumentParser(description='Remote Driving')
#     parser.add_argument('model', type=str,
#     help='Path to model definition json. Model weights should be on the same path.')
#     args = parser.parse_args()
#     with open(args.model, 'r') as jfile:
#         model = model_from_json(jfile.read())
    with open('model2.json', 'r') as jfile:
        model = model_from_json(jfile.read())

    model.compile("adam", "mse")
    weights_file = 'model2.h5'
#     weights_file = './tmp/result1.17-0.07.hdf5'   
    model.load_weights(weights_file)

    
#     if args.image_folder != '':
#         print("Creating image folder at {}".format(args.image_folder))
#         if not os.path.exists(args.image_folder):
#             os.makedirs(args.image_folder)
#         else:
#             shutil.rmtree(args.image_folder)
#             os.makedirs(args.image_folder)
#         print("RECORDING THIS RUN ...")
#     else:
#     print("NOT RECORDING THIS RUN ...")
    
    
    # wrap Flask application with engineio's middleware
    app = socketio.Middleware(sio, app)

    # deploy as an eventlet WSGI server
    eventlet.wsgi.server(eventlet.listen(('', 4567)), app)

Using TensorFlow backend.
(21530) wsgi starting up on http://0.0.0.0:4567
(21530) accepted ('127.0.0.1', 42983)


connect  74a5a5190d3044a7b700b961b07ebce9
-0.15326188504695892 0.3
-0.15326188504695892 0.3
-0.15326188504695892 0.3
-0.15184982120990753 0.3
-0.15184982120990753 0.3
-0.15184982120990753 0.3
-0.1857503354549408 0.3
-0.1559101939201355 0.3
-0.1559101939201355 0.3
-0.08121979981660843 0.3
-0.08121979981660843 0.3
-0.08121979981660843 0.3
-0.04533925652503967 0.3
-0.04533925652503967 0.3
-0.04533925652503967 0.3
-0.06305935233831406 0.3
-0.06305935233831406 0.3
-0.06305935233831406 0.3
-0.24871200323104858 0.3
-0.23629018664360046 0.3
-0.23629018664360046 0.3
-0.0929064005613327 0.3
-0.11623907834291458 0.3
-0.11623907834291458 0.3
-0.09417308866977692 0.3
-0.09417308866977692 0.3
-0.09417308866977692 0.3
-0.03847445547580719 0.3
-0.03847445547580719 0.3
-0.03847445547580719 0.3
-0.04659472033381462 0.3
-0.04659472033381462 0.3
-0.04659472033381462 0.3
-0.056723859161138535 0.3
-0.056723859161138535 0.3
-0.056723859161138535 0.3
0.05004352703690529 0.3
-0.04855238273739815 0.3
-0.0485523

-0.006737048737704754 0.1
0.01891353726387024 0.1
-0.09713290631771088 0.1
-0.09713290631771088 0.1
0.039516955614089966 0.1
0.04970864579081535 0.1
0.04970864579081535 0.1
-0.0025449665263295174 0.1
-0.0025449665263295174 0.3
-0.0025449665263295174 0.3
0.09972701966762543 0.3
0.09972701966762543 0.3
0.09972701966762543 0.3
0.07661285251379013 0.1
0.2335727959871292 0.1
0.2335727959871292 0.1
0.1430150866508484 0.1
0.1430150866508484 0.1
0.1430150866508484 0.1
0.11712467670440674 0.1
0.11712467670440674 0.1
0.11712467670440674 0.1
-0.016012627631425858 0.3
-0.10891549289226532 0.3
-0.10891549289226532 0.3
-0.07993552088737488 0.3
-0.07993552088737488 0.3
-0.07993552088737488 0.3
-0.1827966421842575 0.3
-0.1827966421842575 0.3
-0.1827966421842575 0.3
-0.14774465560913086 0.1
-0.14774465560913086 0.1
-0.14774465560913086 0.1
-0.15828341245651245 0.1
0.05282772704958916 0.1
0.05282772704958916 0.1
0.0458642803132534 0.1
2.5824643671512604e-05 0.1
2.5824643671512604e-05 0.1
0.0988135337829

-0.05508507415652275 0.3
-0.05508507415652275 0.3
0.06210648640990257 0.1
-0.05629340186715126 0.1
-0.05629340186715126 0.1
0.07647670805454254 0.1
0.07647670805454254 0.1
0.07647670805454254 0.1
0.17374145984649658 0.1
0.12415269017219543 0.1
0.12415269017219543 0.1
-0.015026221983134747 0.3
0.05849502608180046 0.3
0.05849502608180046 0.3
0.03745833784341812 0.3
0.13557016849517822 0.3
0.13557016849517822 0.3
0.034130748361349106 0.3
0.03212482109665871 0.1
0.03212482109665871 0.1
-0.027355272322893143 0.1
-0.02774227038025856 0.1
-0.02774227038025856 0.1
-0.09106221795082092 0.1
-0.05140553042292595 0.1
-0.05140553042292595 0.1
-0.1462896168231964 0.1
-0.08662549406290054 0.1
-0.08662549406290054 0.1
-0.1053200364112854 0.3
-0.12241189181804657 0.3
-0.12241189181804657 0.3
-0.16557905077934265 0.3
-0.036804575473070145 0.3
-0.036804575473070145 0.3
-0.0600426085293293 0.3
0.0019311392679810524 0.1
0.0019311392679810524 0.1
0.043715354055166245 0.1
0.043715354055166245 0.1
0.043715354

0.15304748713970184 0.3
-0.017345841974020004 0.3
-0.017345841974020004 0.3
-0.017345841974020004 0.3
0.10324406623840332 0.3
0.10324406623840332 0.3
0.10325753688812256 0.3
-0.02906055375933647 0.1
-0.02906055375933647 0.1
-0.07483121007680893 0.1
-0.03110818937420845 0.1
-0.03110818937420845 0.1
-0.03110818937420845 0.1
-0.16042834520339966 0.1
-0.16042834520339966 0.1
-0.16042834520339966 0.1
-0.03252783045172691 0.3
-0.03252783045172691 0.3
-0.03252783045172691 0.3
-0.05192594975233078 0.3
0.16504226624965668 0.3
0.16504226624965668 0.3
0.10137113928794861 0.3
-0.05336511880159378 0.3
-0.05336511880159378 0.3
0.1151541918516159 0.1
0.08892491459846497 0.1
0.08892491459846497 0.1
0.1985793262720108 0.1
0.1985793262720108 0.1
0.1985793262720108 0.1
0.16488049924373627 0.1
0.16488049924373627 0.1
0.16488049924373627 0.1
0.003900674171745777 0.3
0.003900674171745777 0.1
0.003900674171745777 0.1
0.0029936963692307472 0.3
0.0029936963692307472 0.3
0.0029936963692307472 0.3
-0.02146023511

...

-0.08908920735120773 0.3
-0.08908920735120773 0.3
-0.08908920735120773 0.3
-0.17090532183647156 0.3
-0.17090532183647156 0.3
-0.17090532183647156 0.3
-0.129738450050354 0.3
-0.129738450050354 0.3
-0.129738450050354 0.3
-0.04700060561299324 0.3
-0.04700060561299324 0.3
-0.04700060561299324 0.3
-0.06628870964050293 0.3
-0.06628870964050293 0.3
-0.06628870964050293 0.3
-0.022183019667863846 0.3
0.01217613648623228 0.3
0.01217613648623228 0.3
0.024704236537218094 0.3
0.024704236537218094 0.3
0.024704236537218094 0.3
0.22173163294792175 0.3
0.22173163294792175 0.3
0.22173163294792175 0.3
0.14540785551071167 0.3
0.14540785551071167 0.3
0.14540785551071167 0.3
0.03316456824541092 0.3
0.03316456824541092 0.3
0.03316456824541092 0.3
0.03428741171956062 0.3
0.011647694744169712 0.3
0.011647694744169712 0.3
0.3467365503311157 0.3
0.3467365503311157 0.3
0.3467365503311157 0.3
0.1189185380935669 0.3
0.1189185380935669 0.3
0.1189185380935669 0.3
-0.041823990643024445 0.3
-0.041823990643024445 0.3
-0

0.04578724876046181 0.3
0.04578724876046181 0.3
0.04578724876046181 0.3
0.009243507869541645 0.3
0.009243507869541645 0.3
-0.1326664835214615 0.3
0.05495091900229454 0.3
0.05495091900229454 0.3
0.05495091900229454 0.3
0.08390405774116516 0.3
0.08390405774116516 0.3
0.08390405774116516 0.3
0.0390012226998806 0.3
0.0390012226998806 0.3
-0.0811222493648529 0.3
-0.13617070019245148 0.3
-0.13617070019245148 0.3
-0.13617070019245148 0.3
0.1014326810836792 0.3
0.1014326810836792 0.3
0.1014326810836792 0.3
-0.19466662406921387 0.3
-0.19466662406921387 0.3
-0.19466662406921387 0.3
-0.17907124757766724 0.3
-0.17907124757766724 0.3
-0.17907124757766724 0.3
-0.2545332908630371 0.3
-0.2545332908630371 0.3
-0.10715971887111664 0.3
-0.15507620573043823 0.3
-0.15507620573043823 0.3
-0.15507620573043823 0.3
0.005883974023163319 0.3
0.005883974023163319 0.3
0.005883974023163319 0.3
-0.07584032416343689 0.3
-0.07584032416343689 0.3
-0.07584032416343689 0.3
-0.0636410191655159 0.3
-0.0636410191655159 0.3


-0.12705521285533905 0.1
-0.08932238817214966 0.1
-0.08932238817214966 0.1
-0.053722914308309555 0.1
-0.053722914308309555 0.1
-0.053722914308309555 0.1
-0.17910265922546387 0.1
-0.058225784450769424 0.1
-0.058225784450769424 0.1
-0.05643818899989128 0.1
-0.05643818899989128 0.1
-0.05643818899989128 0.1
-0.11074146628379822 0.1
-0.11074146628379822 0.1
-0.11074146628379822 0.1
-0.13089421391487122 0.1
-0.13089421391487122 0.1
-0.13089421391487122 0.1
0.017582416534423828 0.1
0.017582416534423828 0.1
0.017582416534423828 0.1
-0.03632856905460358 0.1
-0.06439028680324554 0.1
-0.06439028680324554 0.1
-0.020727671682834625 0.1
-0.020727671682834625 0.1
-0.020727671682834625 0.1
-0.021455930545926094 0.1
-0.021455930545926094 0.1
-0.021455930545926094 0.1
0.06439772248268127 0.1
0.06439772248268127 0.1
0.06439772248268127 0.1
0.02847372367978096 0.1
0.02847372367978096 0.1
0.02847372367978096 0.1
0.011082661338150501 0.1
0.011082661338150501 0.1
0.011082661338150501 0.1
0.000333309173583984

0.20002247393131256 0.3
0.20002247393131256 0.3
0.20002247393131256 0.3
-0.1651478260755539 0.3
-0.1651478260755539 0.3
-0.1651478260755539 0.3
-0.2577907145023346 0.3
-0.2577907145023346 0.3
-0.2577907145023346 0.3
0.1070784330368042 0.3
0.1070784330368042 0.3
0.1070784330368042 0.3
-0.061105791479349136 0.3
-0.061105791479349136 0.3
-0.061105791479349136 0.3
-0.0139703918248415 0.3
-0.0139703918248415 0.3
-0.0139703918248415 0.3
-0.026341084390878677 0.3
-0.026341084390878677 0.3
-0.026341084390878677 0.3
0.1665753424167633 0.3
-0.013312171213328838 0.3
-0.013312171213328838 0.3
-0.13681931793689728 0.3
-0.13681931793689728 0.3
-0.13681931793689728 0.3
-0.03413848951458931 0.3
-0.03413848951458931 0.3
-0.03413848951458931 0.3
0.22387628257274628 0.3
0.22387628257274628 0.3
0.22387628257274628 0.3
0.052662547677755356 0.3
0.052662547677755356 0.3
0.017912767827510834 0.3
-0.024712014943361282 0.3
-0.024712014943361282 0.3
-0.024712014943361282 0.3
-0.004208969883620739 0.3
-0.00420896

wsgi exiting
